* Simple Chatbot LLM App
    * will be able to have a conversation
    * Will remember previous interactions: will have memory
    * Will be able to store memory in a Json file

In [2]:
import warnings
from langchain._api import LangChainDeprecationWarning

warnings.simplefilter("ignore", category=LangChainDeprecationWarning)

Connect with the .env file

In [3]:
import os
from dotenv import load_dotenv,find_dotenv

_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

In [6]:
from langchain_openai import ChatOpenAI

chatbot = ChatOpenAI(model="gpt-3.5-turbo")

In [7]:
from langchain_core.messages import HumanMessage

messagesToTheChatbot = [
    HumanMessage(content="My favorite color is blue")
]

## Call to the ChatModel (The LLM)

In [8]:
chatbot.invoke(messagesToTheChatbot)

AIMessage(content='Blue is a calming and soothing color that represents peace, tranquility, and harmony. It is also associated with trust, loyalty, and wisdom. Blue is a versatile color that can be both relaxing and energizing, depending on the shade. Overall, it is a timeless and classic color that is loved by many.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 63, 'prompt_tokens': 12, 'total_tokens': 75, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-1e057082-fcea-46c5-92d0-1436271cc084-0', usage_metadata={'input_tokens': 12, 'output_tokens': 63, 'total_tokens': 75})

## Check if the chatbhot remembers your favourite color

In [10]:
chatbot.invoke(input="what is my favourite color")

AIMessage(content="I'm sorry, I do not know your favorite color. Can you please tell me what your favorite color is?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 12, 'total_tokens': 35, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-3d122514-04c3-4eaf-9c74-f4272e796db9-0', usage_metadata={'input_tokens': 12, 'output_tokens': 23, 'total_tokens': 35})

## Let´s add memory

In [11]:
from langchain import LLMChain
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import HumanMessagePromptTemplate
from langchain_core.prompts import MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from langchain.memory import FileChatMessageHistory

In [12]:
memory = ConversationBufferMemory(
    chat_memory=FileChatMessageHistory('messages.json'),
    memory_key='messages',
    return_messages=True
)

In [13]:
prompt= ChatPromptTemplate(
    input_variables=['content', 'messages'],
    messages=[
        MessagesPlaceholder(variable_name='messages'),
        HumanMessagePromptTemplate.from_template('{content}')
    ]
)

# Memory variable can only be used in legacy chains

In [14]:
chain = LLMChain(
    llm =chatbot,
    prompt=prompt,
    memory=memory
)

In [15]:
chain.invoke({'content': 'hello'})

{'content': 'hello',
 'messages': [],
 'text': 'Hello! How can I assist you today?'}

In [16]:
chain.invoke({'content': 'My name is Javier'})

{'content': 'My name is Javier',
 'messages': [HumanMessage(content='hello', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={})],
 'text': 'Nice to meet you, Javier! How can I help you today?'}

In [17]:
chain.invoke({'content': 'what is my name?'})

{'content': 'what is my name?',
 'messages': [HumanMessage(content='hello', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='My name is Javier', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Nice to meet you, Javier! How can I help you today?', additional_kwargs={}, response_metadata={})],
 'text': 'Your name is Javier. Is there anything else you would like to know or ask?'}